## Izgradnja modela logističke regresije - Lekcija 4

![Infografika: Logistička vs. linearna regresija](../../../../../../2-Regression/4-Logistic/images/linear-vs-logistic.png)

#### **[Kviz prije predavanja](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/15/)**

#### Uvod

U ovoj završnoj lekciji o regresiji, jednoj od osnovnih *klasičnih* tehnika strojnog učenja, pogledat ćemo logističku regresiju. Ovu tehniku koristite za otkrivanje obrazaca kako biste predvidjeli binarne kategorije. Je li ovaj slatkiš čokolada ili nije? Je li ova bolest zarazna ili nije? Hoće li ovaj kupac odabrati ovaj proizvod ili neće?

U ovoj lekciji naučit ćete:

-   Tehnike za logističku regresiju

✅ Produbite svoje razumijevanje rada s ovom vrstom regresije u ovom [modulu za učenje](https://learn.microsoft.com/training/modules/introduction-classification-models/?WT.mc_id=academic-77952-leestott)

## Preduvjet

Radili smo s podacima o bundevama i sada smo dovoljno upoznati s njima da shvatimo da postoji jedna binarna kategorija s kojom možemo raditi: `Color`.

Izgradimo model logističke regresije kako bismo predvidjeli, na temelju nekih varijabli, *koje je boje određena bundeva najvjerojatnije* (narančasta 🎃 ili bijela 👻).

> Zašto govorimo o binarnoj klasifikaciji u lekciji koja se bavi regresijom? Isključivo zbog jezične praktičnosti, budući da je logistička regresija [zapravo metoda klasifikacije](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression), iako se temelji na linearnom pristupu. Saznajte više o drugim načinima klasifikacije podataka u sljedećoj grupi lekcija.

Za ovu lekciju trebat će nam sljedeći paketi:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) je [zbirka R paketa](https://www.tidyverse.org/packages) dizajnirana da učini znanost o podacima bržom, lakšom i zabavnijom!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) okvir je [zbirka paketa](https://www.tidymodels.org/packages/) za modeliranje i strojno učenje.

-   `janitor`: [janitor paket](https://github.com/sfirke/janitor) pruža jednostavne alate za pregled i čišćenje neurednih podataka.

-   `ggbeeswarm`: [ggbeeswarm paket](https://github.com/eclarke/ggbeeswarm) omogućuje stvaranje grafova u stilu "beeswarm" koristeći ggplot2.

Možete ih instalirati pomoću:

`install.packages(c("tidyverse", "tidymodels", "janitor", "ggbeeswarm"))`

Alternativno, skripta ispod provjerava imate li potrebne pakete za dovršavanje ovog modula i instalira ih za vas ako nedostaju.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, ggbeeswarm)


## **Definirajte pitanje**

Za naše potrebe, izrazit ćemo ovo kao binarno: 'Bijelo' ili 'Nije bijelo'. U našem skupu podataka postoji i kategorija 'prugasto', ali ima malo primjera, pa je nećemo koristiti. Ionako nestaje kad uklonimo null vrijednosti iz skupa podataka.

> 🎃 Zanimljiva činjenica, bijele bundeve ponekad nazivamo 'duhovima'. Nisu baš jednostavne za rezbarenje, pa nisu toliko popularne kao narančaste, ali izgledaju zanimljivo! Dakle, mogli bismo također preformulirati naše pitanje kao: 'Duh' ili 'Nije duh'. 👻

## **O logističkoj regresiji**

Logistička regresija razlikuje se od linearne regresije, o kojoj ste ranije učili, na nekoliko važnih načina.

#### **Binarna klasifikacija**

Logistička regresija ne nudi iste značajke kao linearna regresija. Prva nudi predviđanje o `binarnoj kategoriji` ("narančasto ili nije narančasto"), dok je druga sposobna predvidjeti `kontinuirane vrijednosti`, na primjer, s obzirom na podrijetlo bundeve i vrijeme berbe, *koliko će joj cijena porasti*.

![Infografika od Dasani Madipalli](../../../../../../2-Regression/4-Logistic/images/pumpkin-classifier.png)

### Ostale klasifikacije

Postoje i druge vrste logističke regresije, uključujući multinomijalnu i ordinalnu:

- **Multinomijalna**, koja uključuje više od jedne kategorije - "Narančasto, Bijelo i Prugasto".

- **Ordinalna**, koja uključuje poredane kategorije, korisno ako želimo logično poredati naše ishode, poput bundeva koje su poredane prema ograničenom broju veličina (mini,sm,med,lg,xl,xxl).

![Multinomijalna vs ordinalna regresija](../../../../../../2-Regression/4-Logistic/images/multinomial-vs-ordinal.png)

#### **Varijable NE MORAJU biti povezane**

Sjećate se kako je linearna regresija bolje funkcionirala s više povezanih varijabli? Logistička regresija je suprotna - varijable ne moraju biti povezane. To odgovara ovom skupu podataka koji ima relativno slabe korelacije.

#### **Potrebno je puno čistih podataka**

Logistička regresija daje točnije rezultate ako koristite više podataka; naš mali skup podataka nije optimalan za ovaj zadatak, pa to imajte na umu.

✅ Razmislite o vrstama podataka koje bi bile prikladne za logističku regresiju

## Vježba - očistite podatke

Prvo, malo očistite podatke, uklonite null vrijednosti i odaberite samo neke od stupaca:

1. Dodajte sljedeći kod:


In [ ]:
# Load the core tidyverse packages
library(tidyverse)

# Import the data and clean column names
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv") %>% 
  clean_names()

# Select desired columns
pumpkins_select <- pumpkins %>% 
  select(c(city_name, package, variety, origin, item_size, color)) 

# Drop rows containing missing values and encode color as factor (category)
pumpkins_select <- pumpkins_select %>% 
  drop_na() %>% 
  mutate(color = factor(color))

# View the first few rows
pumpkins_select %>% 
  slice_head(n = 5)


Uvijek možete zaviriti u svoj novi dataframe koristeći funkciju [*glimpse()*](https://pillar.r-lib.org/reference/glimpse.html) kao što je prikazano dolje:


In [ ]:
pumpkins_select %>% 
  glimpse()


Hajdemo potvrditi da ćemo zapravo raditi na problemu binarne klasifikacije:


In [ ]:
# Subset distinct observations in outcome column
pumpkins_select %>% 
  distinct(color)


### Vizualizacija - kategorijalni graf
Do sada ste ponovno učitali podatke o bundevama i očistili ih kako biste sačuvali skup podataka koji sadrži nekoliko varijabli, uključujući Boju. Vizualizirajmo dataframe u bilježnici koristeći ggplot biblioteku.

Biblioteka ggplot nudi zanimljive načine za vizualizaciju vaših podataka. Na primjer, možete usporediti distribucije podataka za svaku Sortu i Boju u kategorijalnom grafu.

1. Kreirajte takav graf koristeći funkciju geombar, koristeći naše podatke o bundevama, i specificirajte mapiranje boja za svaku kategoriju bundeva (narančasta ili bijela):


In [ ]:
# Specify colors for each value of the hue variable
palette <- c(ORANGE = "orange", WHITE = "wheat")

# Create the bar plot
ggplot(pumpkins_select, aes(y = variety, fill = color)) +
  geom_bar(position = "dodge") +
  scale_fill_manual(values = palette) +
  labs(y = "Variety", fill = "Color") +
  theme_minimal()

Promatrajući podatke, možete vidjeti kako se podaci o Boji odnose na Sortu.

✅ S obzirom na ovaj kategorijski grafikon, koja zanimljiva istraživanja možete zamisliti?


### Obrada podataka: kodiranje značajki

Naš skup podataka o bundevama sadrži tekstualne vrijednosti u svim stupcima. Rad s kategorijskim podacima je intuitivan za ljude, ali ne i za strojeve. Algoritmi strojnog učenja bolje funkcioniraju s brojčanim podacima. Zato je kodiranje vrlo važan korak u fazi obrade podataka, jer nam omogućuje pretvaranje kategorijskih podataka u brojčane, bez gubitka informacija. Dobro kodiranje vodi do izgradnje dobrog modela.

Za kodiranje značajki postoje dvije glavne vrste kodera:

1. **Ordinalni koder**: dobro odgovara za ordinalne varijable, koje su kategorijske varijable čiji podaci slijede logički redoslijed, poput stupca `item_size` u našem skupu podataka. Stvara mapiranje tako da je svaka kategorija predstavljena brojem, koji odgovara redoslijedu kategorije u stupcu.

2. **Kategorijski koder**: dobro odgovara za nominalne varijable, koje su kategorijske varijable čiji podaci ne slijede logički redoslijed, poput svih značajki koje nisu `item_size` u našem skupu podataka. Ovo je kodiranje s jednom vrućom vrijednošću (one-hot encoding), što znači da je svaka kategorija predstavljena binarnim stupcem: kodirana varijabla jednaka je 1 ako bundeva pripada toj sorti, a 0 inače.

Tidymodels pruža još jedan koristan paket: [recipes](https://recipes.tidymodels.org/) - paket za obradu podataka. Definirat ćemo `recipe` koji specificira da svi stupci prediktora trebaju biti kodirani u skup cijelih brojeva, `prep` za procjenu potrebnih količina i statistika potrebnih za bilo koje operacije, i na kraju `bake` za primjenu izračuna na nove podatke.

> Obično se recipes koristi kao preprocesor za modeliranje, gdje definira koje korake treba primijeniti na skup podataka kako bi bio spreman za modeliranje. U tom slučaju **toplo se preporučuje** korištenje `workflow()` umjesto ručnog procjenjivanja recepta pomoću prep i bake. Sve ćemo to vidjeti uskoro.
>
> Međutim, za sada koristimo recipes + prep + bake kako bismo specificirali koje korake treba primijeniti na skup podataka kako bi bio spreman za analizu podataka, a zatim izdvojili prethodno obrađene podatke s primijenjenim koracima.


In [ ]:
# Preprocess and extract data to allow some data analysis
baked_pumpkins <- recipe(color ~ ., data = pumpkins_select) %>%
  # Define ordering for item_size column
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  # Convert factors to numbers using the order defined above (Ordinal encoding)
  step_integer(item_size, zero_based = F) %>%
  # Encode all other predictors using one hot encoding
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE) %>%
  prep(data = pumpkin_select) %>%
  bake(new_data = NULL)

# Display the first few rows of preprocessed data
baked_pumpkins %>% 
  slice_head(n = 5)


✅ Koje su prednosti korištenja ordinalnog kodera za stupac Veličina artikla?

### Analizirajte odnose između varijabli

Sada kada smo unaprijed obradili naše podatke, možemo analizirati odnose između značajki i oznake kako bismo stekli ideju o tome koliko će model biti uspješan u predviđanju oznake na temelju značajki. Najbolji način za provođenje ove vrste analize je vizualizacija podataka. 
Ponovno ćemo koristiti funkciju ggplot geom_boxplot_ kako bismo vizualizirali odnose između Veličine artikla, Sorte i Boje u kategorijalnom grafu. Za bolju vizualizaciju podataka koristit ćemo kodirani stupac Veličina artikla i nekodirani stupac Sorta.


In [ ]:
# Define the color palette
palette <- c(ORANGE = "orange", WHITE = "wheat")

# We need the encoded Item Size column to use it as the x-axis values in the plot
pumpkins_select_plot<-pumpkins_select
pumpkins_select_plot$item_size <- baked_pumpkins$item_size

# Create the grouped box plot
ggplot(pumpkins_select_plot, aes(x = `item_size`, y = color, fill = color)) +
  geom_boxplot() +
  facet_grid(variety ~ ., scales = "free_x") +
  scale_fill_manual(values = palette) +
  labs(x = "Item Size", y = "") +
  theme_minimal() +
  theme(strip.text = element_text(size = 12)) +
  theme(axis.text.x = element_text(size = 10)) +
  theme(axis.title.x = element_text(size = 12)) +
  theme(axis.title.y = element_blank()) +
  theme(legend.position = "bottom") +
  guides(fill = guide_legend(title = "Color")) +
  theme(panel.spacing = unit(0.5, "lines"))+
  theme(strip.text.y = element_text(size = 4, hjust = 0)) 


#### Koristite swarm plot

Budući da je Color binarna kategorija (Bijela ili Ne), potrebna je '[specijalizirani pristup](https://github.com/rstudio/cheatsheets/blob/main/data-visualization.pdf)' za vizualizaciju.

Isprobajte `swarm plot` kako biste prikazali distribuciju boje u odnosu na item_size.

Koristit ćemo [ggbeeswarm paket](https://github.com/eclarke/ggbeeswarm) koji pruža metode za kreiranje grafova u stilu "beeswarm" koristeći ggplot2. Beeswarm grafovi omogućuju prikaz točaka koje bi se inače preklapale tako da se smjeste jedna pored druge.


In [ ]:
# Create beeswarm plots of color and item_size
baked_pumpkins %>% 
  mutate(color = factor(color)) %>% 
  ggplot(mapping = aes(x = color, y = item_size, color = color)) +
  geom_quasirandom() +
  scale_color_brewer(palette = "Dark2", direction = -1) +
  theme(legend.position = "none")


Sad kad imamo ideju o odnosu između binarnih kategorija boje i šire skupine veličina, istražimo logističku regresiju kako bismo odredili vjerojatnu boju određene bundeve.

## Izgradite svoj model

Odaberite varijable koje želite koristiti u svom klasifikacijskom modelu i podijelite podatke na skupove za treniranje i testiranje. [rsample](https://rsample.tidymodels.org/), paket u Tidymodels, pruža infrastrukturu za učinkovito dijeljenje i ponovnu uzorkovanje podataka:


In [ ]:
# Split data into 80% for training and 20% for testing
set.seed(2056)
pumpkins_split <- pumpkins_select %>% 
  initial_split(prop = 0.8)

# Extract the data in each split
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)

# Print out the first 5 rows of the training set
pumpkins_train %>% 
  slice_head(n = 5)


🙌 Sada smo spremni trenirati model prilagođavanjem značajki za treniranje oznaci za treniranje (boji).

Počet ćemo stvaranjem recepta koji određuje korake predobrade koje treba provesti na našim podacima kako bi ih pripremili za modeliranje, tj. kodiranjem kategorijalnih varijabli u skup cijelih brojeva. Baš kao `baked_pumpkins`, kreiramo `pumpkins_recipe`, ali ne koristimo `prep` i `bake` jer će biti uključeni u tijek rada, što ćete vidjeti za nekoliko koraka.

Postoji prilično mnogo načina za specificiranje logističke regresije u Tidymodels. Pogledajte `?logistic_reg()`. Za sada ćemo specificirati model logističke regresije putem zadane `stats::glm()` pogonske jedinice.


In [ ]:
# Create a recipe that specifies preprocessing steps for modelling
pumpkins_recipe <- recipe(color ~ ., data = pumpkins_train) %>% 
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  step_integer(item_size, zero_based = F) %>%  
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE)

# Create a logistic model specification
log_reg <- logistic_reg() %>% 
  set_engine("glm") %>% 
  set_mode("classification")


Sada kada imamo recept i specifikaciju modela, trebamo pronaći način da ih spojimo u objekt koji će prvo obraditi podatke (priprema + pečenje u pozadini), prilagoditi model na obrađenim podacima i također omogućiti potencijalne aktivnosti post-obrade.

U Tidymodels-u, ovaj praktičan objekt naziva se [`workflow`](https://workflows.tidymodels.org/) i praktično drži vaše komponente modeliranja.


In [ ]:
# Bundle modelling components in a workflow
log_reg_wf <- workflow() %>% 
  add_recipe(pumpkins_recipe) %>% 
  add_model(log_reg)

# Print out the workflow
log_reg_wf


Nakon što je tijek rada *specificiran*, model se može `trenirati` koristeći funkciju [`fit()`](https://tidymodels.github.io/parsnip/reference/fit.html). Tijek rada će procijeniti recept i unaprijed obraditi podatke prije treniranja, tako da to nećemo morati ručno raditi koristeći prep i bake.


In [ ]:
# Train the model
wf_fit <- log_reg_wf %>% 
  fit(data = pumpkins_train)

# Print the trained workflow
wf_fit


Model ispisuje koeficijente naučene tijekom treniranja.

Sada kada smo model istrenirali koristeći podatke za treniranje, možemo napraviti predikcije na testnim podacima koristeći [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Počnimo tako da koristimo model za predviđanje oznaka za naš testni skup i vjerojatnosti za svaku oznaku. Kada je vjerojatnost veća od 0.5, predviđena klasa je `WHITE`, inače je `ORANGE`.


In [ ]:
# Make predictions for color and corresponding probabilities
results <- pumpkins_test %>% select(color) %>% 
  bind_cols(wf_fit %>% 
              predict(new_data = pumpkins_test)) %>%
  bind_cols(wf_fit %>%
              predict(new_data = pumpkins_test, type = "prob"))

# Compare predictions
results %>% 
  slice_head(n = 10)


Vrlo dobro! Ovo pruža dodatne uvide u to kako logistička regresija funkcionira.

### Bolje razumijevanje putem matrice zabune

Uspoređivanje svake predikcije s odgovarajućom "stvarnom" vrijednošću nije baš učinkovit način za određivanje koliko dobro model predviđa. Srećom, Tidymodels ima još nekoliko trikova u rukavu: [`yardstick`](https://yardstick.tidymodels.org/) - paket koji se koristi za mjerenje učinkovitosti modela pomoću metrika izvedbe.

Jedna od metrika izvedbe povezanih s problemima klasifikacije je [`matrica zabune`](https://wikipedia.org/wiki/Confusion_matrix). Matrica zabune opisuje koliko dobro klasifikacijski model funkcionira. Matrica zabune prikazuje koliko je primjera u svakoj klasi model ispravno klasificirao. U našem slučaju, pokazat će vam koliko je narančastih bundeva klasificirano kao narančaste, a koliko bijelih bundeva kao bijele; matrica zabune također pokazuje koliko je primjera klasificirano u **pogrešne** kategorije.

Funkcija [**`conf_mat()`**](https://tidymodels.github.io/yardstick/reference/conf_mat.html) iz paketa yardstick izračunava ovu križnu tablicu promatranih i predviđenih klasa.


In [ ]:
# Confusion matrix for prediction results
conf_mat(data = results, truth = color, estimate = .pred_class)


Hajdemo interpretirati matricu konfuzije. Naš model treba klasificirati bundeve u dvije binarne kategorije, kategoriju `bijela` i kategoriju `ne-bijela`.

-   Ako vaš model predvidi bundevu kao bijelu, a ona stvarno pripada kategoriji 'bijela', to nazivamo `istinitim pozitivnim` (true positive), prikazano gornjim lijevim brojem.

-   Ako vaš model predvidi bundevu kao ne-bijelu, a ona stvarno pripada kategoriji 'bijela', to nazivamo `lažno negativnim` (false negative), prikazano donjim lijevim brojem.

-   Ako vaš model predvidi bundevu kao bijelu, a ona stvarno pripada kategoriji 'ne-bijela', to nazivamo `lažno pozitivnim` (false positive), prikazano gornjim desnim brojem.

-   Ako vaš model predvidi bundevu kao ne-bijelu, a ona stvarno pripada kategoriji 'ne-bijela', to nazivamo `istinitim negativnim` (true negative), prikazano donjim desnim brojem.

| Istina |
|:-----:|


|               |        |       |
|---------------|--------|-------|
| **Predviđeno** | BIJELA | NARANČASTA |
| BIJELA        | TP     | FP    |
| NARANČASTA    | FN     | TN    |

Kao što ste možda pretpostavili, poželjno je imati veći broj istinitih pozitivnih i istinitih negativnih te manji broj lažno pozitivnih i lažno negativnih, što implicira da model bolje radi.

Matrica konfuzije je korisna jer omogućuje izračun drugih metrika koje nam mogu pomoći u boljoj procjeni performansi modela klasifikacije. Pogledajmo neke od njih:

🎓 Preciznost: `TP/(TP + FP)` definirana kao udio predviđenih pozitivnih koji su stvarno pozitivni. Također se naziva [pozitivna prediktivna vrijednost](https://en.wikipedia.org/wiki/Positive_predictive_value "Positive predictive value").

🎓 Odziv: `TP/(TP + FN)` definiran kao udio pozitivnih rezultata u odnosu na broj uzoraka koji su stvarno pozitivni. Također poznat kao `osjetljivost`.

🎓 Specifičnost: `TN/(TN + FP)` definirana kao udio negativnih rezultata u odnosu na broj uzoraka koji su stvarno negativni.

🎓 Točnost: `TP + TN/(TP + TN + FP + FN)` Postotak oznaka koje su točno predviđene za uzorak.

🎓 F mjera: Ponderirani prosjek preciznosti i odziva, gdje je najbolji rezultat 1, a najgori 0.

Izračunajmo ove metrike!


In [ ]:
# Combine metric functions and calculate them all at once
eval_metrics <- metric_set(ppv, recall, spec, f_meas, accuracy)
eval_metrics(data = results, truth = color, estimate = .pred_class)


## Vizualiziraj ROC krivulju ovog modela

Napravimo još jednu vizualizaciju kako bismo vidjeli takozvanu [`ROC krivulju`](https://en.wikipedia.org/wiki/Receiver_operating_characteristic):


In [ ]:
# Make a roc_curve
results %>% 
  roc_curve(color, .pred_ORANGE) %>% 
  autoplot()


ROC krivulje često se koriste za prikaz rezultata klasifikatora u smislu njegovih pravih naspram lažnih pozitivnih rezultata. ROC krivulje obično prikazuju `True Positive Rate`/osjetljivost na Y osi i `False Positive Rate`/1-specifičnost na X osi. Dakle, važni su nagib krivulje i prostor između središnje linije i krivulje: želite krivulju koja brzo ide prema gore i prelazi liniju. U našem slučaju, postoje lažno pozitivni rezultati na početku, a zatim linija pravilno ide prema gore i prelazi.

Na kraju, koristimo `yardstick::roc_auc()` za izračunavanje stvarnog područja ispod krivulje (Area Under the Curve). Jedan od načina tumačenja AUC-a je kao vjerojatnost da model rangira slučajni pozitivni primjer više od slučajnog negativnog primjera.


In [ ]:
# Calculate area under curve
results %>% 
  roc_auc(color, .pred_ORANGE)


Rezultat je oko `0.975`. S obzirom na to da AUC varira od 0 do 1, želite postići visok rezultat, jer model koji je 100% točan u svojim predviđanjima ima AUC od 1; u ovom slučaju, model je *prilično dobar*.

U budućim lekcijama o klasifikacijama naučit ćete kako poboljšati rezultate svog modela (kao što je rješavanje problema neuravnoteženih podataka u ovom slučaju).

## 🚀Izazov

Logistička regresija krije još mnogo toga! No, najbolji način za učenje je eksperimentiranje. Pronađite skup podataka koji se može analizirati ovom metodom i izgradite model na temelju njega. Što ste naučili? savjet: isprobajte [Kaggle](https://www.kaggle.com/search?q=logistic+regression+datasets) za zanimljive skupove podataka.

## Pregled i samostalno učenje

Pročitajte prvih nekoliko stranica [ovog rada sa Stanforda](https://web.stanford.edu/~jurafsky/slp3/5.pdf) o nekim praktičnim primjenama logističke regresije. Razmislite o zadacima koji su bolje prilagođeni jednoj ili drugoj vrsti regresije koje smo do sada proučavali. Što bi najbolje funkcioniralo?



---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden pomoću AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati autoritativnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane ljudskog prevoditelja. Ne preuzimamo odgovornost za bilo kakve nesporazume ili pogrešne interpretacije koje proizlaze iz korištenja ovog prijevoda.
